In [1]:
#
# As variáveis de ambiente e globais abaixo precisam ser configuradas
# antes de rodar o notebook.
#

# Caminho para a chave do service account do GCP
# Veja: https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries
%env GOOGLE_APPLICATION_CREDENTIALS=./google-cloud-key.json

# Credenciais para criancao de user
%env FULL_NAME=Marcela Barella
%env EMAIL=marcela.barella@hotmail.com

BQ_PROJECT="galvanic-smoke-385402"
BQ_DATASET="cadastro_produto"
BQ_TABLE="kh_data_engineer_teste_marcelabarella"

env: GOOGLE_APPLICATION_CREDENTIALS=./google-cloud-key.json
env: FULL_NAME=Marcela Barella
env: EMAIL=marcela.barella@hotmail.com


In [2]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

You should consider upgrading via the '/Users/grelinhodediamante/Projects/karhub/venv/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 22.4 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 190 kB 92.9 MB/s eta 0:00:01
You should consider upgrading via the '/Users/grelinhodediamante/Projects/karhub/venv/bin/python3 -m pip install --upgrade pip' command.


In [3]:
%run -i 'karhub_extraction.py'

In [5]:
import json
import pandas as pd

In [6]:
with open('./cars_data.json', 'r') as file:
    raw_data = json.load(file)

In [35]:
df_cars = pd.DataFrame.from_dict(raw_data)

In [36]:
df_cars

,Fabricante,Código Fabricante,Marca,Modelo,Ano,Complemento
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P | 1.0 MPI 8V - 4P | 1.0 ...
1,KarHub,FO0352,FORD,FIESTA,1997.0,1.0 MPI 8V - 2P | 1.3 MPI 8V CLX 2P | 1.3 MP...
2,KarHub,FO0352,FORD,FIESTA,1998.0,1.0 MPI 8V CLASS 2P | 1.0 MPI 8V CLASS 4P | ...
3,KarHub,FO0353,FORD,KA,1997.0,1.0i 8V - 2P | 1.3 MPI 8V CLX 2P | 1.0 MPI 8...
4,KarHub,FO0353,FORD,KA,1998.0,1.0i 8V - 2P | 1.3 MPI 8V CLX 2P | 1.0 MPI 8...
...,...,...,...,...,...,...
4210,KarHub,I-VW0468,AUDI,A3,2004.0,1.8 TURBO 20V 180CV - 4P | 1.6 8V - 2P | 1.8...
4211,KarHub,I-VW0468,AUDI,A3,2005.0,1.6 8V - 4P | 1.8 20V - 4P | 1.8 20V - 2P |...
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 20V - 4P | 1.6 8V - 4P | 1.8 TURBO 20V 1...
4213,KarHub,I-CH0512,None,None,NaN,None


In [39]:
df_cars['Fabricante'].fillna('', inplace=True)
df_cars['Marca'].fillna('', inplace=True)
df_cars['Modelo'].fillna('', inplace=True)
df_cars['Ano'].fillna(0.0, inplace=True)
df_cars['Complemento'].fillna('', inplace=True)

In [40]:
df_cars = df_cars.assign(Complemento=df_cars['Complemento'].str.split('|'))

In [41]:
df_cars = df_cars.explode('Complemento')

In [42]:
df_cars = df_cars.rename(columns={'Código Fabricante': 'Código'})
df_cars = df_cars.drop(['Fabricante'], axis=1)

In [43]:
df_cars

,Código,Marca,Modelo,Ano,Complemento
0,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P
0,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V - 4P
0,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V - 2P
0,FO0352,FORD,FIESTA,1996.0,1.3 MPI 8V CLX 4P
0,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 4P
...,...,...,...,...,...
4212,I-VW0468,AUDI,A3,2006.0,1.8 TURBO 20V 150CV - 4P
4212,I-VW0468,AUDI,A3,2006.0,1.8 20V - 2P
4212,I-VW0468,AUDI,A3,2006.0,1.8 TURBO 20V 180CV - 4P
4213,I-CH0512,,,0.0,


In [44]:
autoparts_df = pd.read_excel('karhub_autoparts_1.xlsx')

In [66]:
autoparts_df

,Nome SKU,Fabricante,Código,Composição,Categoria,Nome Atributo,Valor Atributo
0,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Largura (cm),18.0
1,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Altura (cm),9.0
2,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Comprimento (cm),8.0
3,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Peso Bruto (Kg),0.600
4,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Peso Líquido (Kg),0.600
...,...,...,...,...,...,...,...
3313,Mola de Suspensão Dianteira Helicoidal,KarHub,I-VW0406,UNITARY,Mola Helicoidal,Largura (cm),40
3314,Mola de Suspensão Dianteira Helicoidal,KarHub,I-VW0406,UNITARY,Mola Helicoidal,Altura (cm),25
3315,Mola de Suspensão Dianteira Helicoidal,KarHub,I-VW0406,UNITARY,Mola Helicoidal,Comprimento (cm),25
3316,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0605,UNITARY,Mola Helicoidal,Largura (cm),40


In [95]:
pivoted_autoparts_df = autoparts_df.pivot_table(
    index='Código',
    columns='Nome Atributo',
    values='Valor Atributo',
    aggfunc='first'
)

In [96]:
pivoted_autoparts_df

Nome Atributo,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição
Código,,,,,,,,
0054,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Traseira
07402767.0,25,25,40,NaN,"10,75",4.5,2.0,Traseira
CH-0017,25,25,40,NaN,"7,3",4.5,2.0,Traseira
CH-0029,25,25,40,NaN,7,4.5,2.0,Dianteira
CH-0033,25,25,40,NaN,"7,5",4.5,2.0,Traseira
...,...,...,...,...,...,...,...,...
vw0425,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Traseira
vw0446,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Traseira
vw0474,9.0,8.0,18.0,NaN,NaN,0.600,0.600,Dianteira


In [97]:
pivoted_autoparts_df['Altura (cm)'] = pivoted_autoparts_df['Altura (cm)'].str.replace(',','.')
pivoted_autoparts_df['Altura (cm)'].fillna(0.0, inplace=True)

pivoted_autoparts_df['Comprimento (cm)'] = pivoted_autoparts_df['Comprimento (cm)'].str.replace(',','.')
pivoted_autoparts_df['Comprimento (cm)'].fillna(0.0, inplace=True)

pivoted_autoparts_df['Largura (cm)'] = pivoted_autoparts_df['Largura (cm)'].str.replace(',','.')
pivoted_autoparts_df['Largura (cm)'].fillna(0.0, inplace=True)

pivoted_autoparts_df['Motor'].fillna('', inplace=True)

pivoted_autoparts_df['Número de Espirais'] = pivoted_autoparts_df['Número de Espirais'].str.replace(',','.')
pivoted_autoparts_df['Número de Espirais'].fillna(0.0, inplace=True)

pivoted_autoparts_df['Peso Bruto (Kg)'] = pivoted_autoparts_df['Peso Bruto (Kg)'].str.replace(',','.')
pivoted_autoparts_df['Peso Bruto (Kg)'].fillna(0.0, inplace=True)

pivoted_autoparts_df['Peso Líquido (Kg)'] = pivoted_autoparts_df['Peso Líquido (Kg)'].str.replace(',','.')
pivoted_autoparts_df['Peso Líquido (Kg)'].fillna(0.0, inplace=True)

pivoted_autoparts_df['Posição'].fillna('', inplace=True)

In [98]:
pivoted_autoparts_df

Nome Atributo,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição
Código,,,,,,,,
0054,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
07402767.0,25,25,40,,10.75,4.5,2.0,Traseira
CH-0017,25,25,40,,7.3,4.5,2.0,Traseira
CH-0029,25,25,40,,7,4.5,2.0,Dianteira
CH-0033,25,25,40,,7.5,4.5,2.0,Traseira
...,...,...,...,...,...,...,...,...
vw0425,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
vw0446,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
vw0474,9.0,8.0,18.0,,0.0,0.600,0.600,Dianteira


In [99]:
pivoted_autoparts_df_reset = pivoted_autoparts_df.reset_index()

In [100]:
pivoted_autoparts_df_reset

Nome Atributo,Código,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição
0,0054,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
1,07402767.0,25,25,40,,10.75,4.5,2.0,Traseira
2,CH-0017,25,25,40,,7.3,4.5,2.0,Traseira
3,CH-0029,25,25,40,,7,4.5,2.0,Dianteira
4,CH-0033,25,25,40,,7.5,4.5,2.0,Traseira
...,...,...,...,...,...,...,...,...,...
475,vw0425,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
476,vw0446,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
477,vw0474,9.0,8.0,18.0,,0.0,0.600,0.600,Dianteira
478,vw0478,9.0,8.0,18.0,,0.0,0.600,0.600,Dianteira


In [101]:
autoparts_df = autoparts_df.merge(pivoted_autoparts_df_reset, on='Código', how='inner')

In [102]:
autoparts_df

,Nome SKU,Fabricante,Código,Composição,Categoria,Nome Atributo,Valor Atributo,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição
0,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Largura (cm),18.0,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
1,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Altura (cm),9.0,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
2,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Comprimento (cm),8.0,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
3,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Peso Bruto (Kg),0.600,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
4,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Peso Líquido (Kg),0.600,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0611,UNITARY,Mola Helicoidal,Comprimento (cm),25,25,25,40,,7.1,4.5,2.0,Dianteira
3314,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0611,UNITARY,Mola Helicoidal,Peso Bruto (Kg),4.5,25,25,40,,7.1,4.5,2.0,Dianteira
3315,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0611,UNITARY,Mola Helicoidal,Peso Líquido (Kg),2.0,25,25,40,,7.1,4.5,2.0,Dianteira
3316,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0611,UNITARY,Mola Helicoidal,Posição,Dianteira,25,25,40,,7.1,4.5,2.0,Dianteira


In [116]:
alias_df = pd.read_csv('karhub-alias.csv')

In [117]:
alias_df

,Fabricante,Código Fabricante,Alias
0,KarHub,FO0352,fo0352
1,KarHub,FO0353,fo0353
2,KarHub,FT0417,ft0417
3,KarHub,VW0127,vw0127
4,KarHub,PE0452,pe0452
...,...,...,...
1245,KarHub,I-VW0468,VW0468
1246,KarHub,I-CH0512,I-CH0512
1247,KarHub,I-CH0512,ich0512
1248,KarHub,I-MI0611,I-MI0611


In [118]:
alias_df.fillna('', inplace=True)

In [119]:
products_df = autoparts_df.merge(alias_df, left_on='Código', right_on='Alias')

In [120]:
products_df

,Nome SKU,Fabricante_x,Código,Composição,Categoria,Nome Atributo,Valor Atributo,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição,Fabricante_y,Código Fabricante,Alias
0,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Largura (cm),18.0,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352,fo0352
1,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Altura (cm),9.0,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352,fo0352
2,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Comprimento (cm),8.0,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352,fo0352
3,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Peso Bruto (Kg),0.600,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352,fo0352
4,Mola da Suspensão Traseira,KarHub,fo0352,UNITARY,Mola Helicoidal,Peso Líquido (Kg),0.600,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352,fo0352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0611,UNITARY,Mola Helicoidal,Comprimento (cm),25,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611,I-MI0611
3296,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0611,UNITARY,Mola Helicoidal,Peso Bruto (Kg),4.5,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611,I-MI0611
3297,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0611,UNITARY,Mola Helicoidal,Peso Líquido (Kg),2.0,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611,I-MI0611
3298,Mola de Suspensão Dianteira Helicoidal,KarHub,I-MI0611,UNITARY,Mola Helicoidal,Posição,Dianteira,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611,I-MI0611


In [121]:
products_df = products_df.drop(['Fabricante_x', 'Código', 'Alias', 'Nome Atributo', 'Valor Atributo'], axis=1)

In [122]:
products_df

,Nome SKU,Composição,Categoria,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição,Fabricante_y,Código Fabricante
0,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352
1,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352
2,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352
3,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352
4,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611
3296,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611
3297,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611
3298,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611


In [130]:
products_df = products_df.rename(columns={'Fabricante_y': 'Fabricante', 'Código Fabricante': 'Código'})

In [131]:
products_df

,Nome SKU,Composição,Categoria,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição,Fabricante,Código
0,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352
1,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352
2,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352
3,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352
4,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.600,0.600,Traseira,KarHub,FO0352
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611
3296,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611
3297,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611
3298,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25,25,40,,7.1,4.5,2.0,Dianteira,KarHub,I-MI0611


In [132]:
cars_products_df = df_cars.merge(products_df, on='Código', how='inner')

In [133]:
cars_products_df = cars_products_df.rename(columns={'Código': 'codigo', 'Composição': 'composicao', \
                                                    'Marca': 'marca', 'Modelo': 'modelo', \
                                                    'Ano': 'ano', 'Complemento': 'complemento', \
                                                    'Nome SKU': 'nome_sku', 'Categoria': 'categoria', \
                                                    'Altura (cm)': 'altura_cm', 'Comprimento (cm)': 'comprimento_cm', \
                                                    'Largura (cm)': 'largura_cm', 'Motor': 'motor', \
                                                   'Número de Espirais': 'numero_de_espirais', \
                                                    'Peso Bruto (Kg)': 'peso_bruto_kg', \
                                                    'Peso Líquido (Kg)': 'peso_liquido_kg', \
                                                    'Posição': 'posicao', 'Fabricante': 'fabricante'\
                                                   })

In [137]:
cars_products_df = cars_products_df.astype({'ano': float, 'altura_cm': float, 'comprimento_cm': float, \
                                           'largura_cm': float, 'numero_de_espirais': float, \
                                           'peso_bruto_kg': float, 'peso_liquido_kg': float})

In [138]:
cars_products_df

,codigo,marca,modelo,ano,complemento,nome_sku,composicao,categoria,altura_cm,comprimento_cm,largura_cm,motor,numero_de_espirais,peso_bruto_kg,peso_liquido_kg,posicao,fabricante
0,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.6,0.6,Traseira,KarHub
1,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.6,0.6,Traseira,KarHub
2,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.6,0.6,Traseira,KarHub
3,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.6,0.6,Traseira,KarHub
4,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P,Mola da Suspensão Traseira,UNITARY,Mola Helicoidal,9.0,8.0,18.0,,0.0,0.6,0.6,Traseira,KarHub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187032,I-MI0611,,,0.0,,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25.0,25.0,40.0,,7.1,4.5,2.0,Dianteira,KarHub
187033,I-MI0611,,,0.0,,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25.0,25.0,40.0,,7.1,4.5,2.0,Dianteira,KarHub
187034,I-MI0611,,,0.0,,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25.0,25.0,40.0,,7.1,4.5,2.0,Dianteira,KarHub
187035,I-MI0611,,,0.0,,Mola de Suspensão Dianteira Helicoidal,UNITARY,Mola Helicoidal,25.0,25.0,40.0,,7.1,4.5,2.0,Dianteira,KarHub


In [139]:
from google.cloud import bigquery

def save_to_bigquery(df):
    bq_table_id = f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}'

    table_config = bigquery.LoadJobConfig(
        # schema=[
        #     # Força a tabela a ter o campo release_date como campo de data ao invés de datetime
        #     # já que os dados não possuem informação de tempo
        #     bigquery.SchemaField("release_date", bigquery.enums.SqlTypeNames.DATE)
        # ],
        # Sobescrever a tabela inteira quando rodar o script
        write_disposition="WRITE_TRUNCATE",
    )

    client = bigquery.Client()
    client.load_table_from_dataframe(df, bq_table_id, job_config=table_config)

In [140]:
save_to_bigquery(cars_products_df)